In [403]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database connection

In [404]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [405]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [406]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

In [407]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas sin saltos
pd.set_option('display.width', 1000)    

## Extraction

In [408]:
cliente = pd.read_sql_table('dim_cliente', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
hora = pd.read_sql_table('dim_hora', etl_conn)
fecha = pd.read_sql_table('dim_fecha', etl_conn)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [409]:
total_nan = servicio.isna().sum()
print(total_nan)

id                                    0
descripcion                           0
nombre_solicitante                    0
fecha_solicitud                       0
hora_solicitud                        0
fecha_deseada                         6
hora_deseada                          0
nombre_recibe                         0
telefono_recibe                       0
descripcion_pago                    260
ida_y_regreso                         0
activo                                0
novedades                           336
cliente_id                            0
destino_id                            0
mensajero_id                        727
origen_id                             0
tipo_pago_id                          0
tipo_servicio_id                      0
tipo_vehiculo_id                      0
usuario_id                            0
prioridad                             0
ciudad_destino_id                     0
ciudad_origen_id                      0
hora_visto_por_mensajero          28428


## Transformation

In [410]:
print(hora.columns)
tiempo_unique = hora[["hora", "key_hora"]].drop_duplicates(subset=["hora"])


# Convertir 'hora_solicitud' a string en el formato HH:MM:SS
# Convertir 'hora_solicitud' a datetime
servicio["hora_solicitud"] = pd.to_datetime(servicio["hora_solicitud"], format='%H:%M:%S')

tiempo_unique["hora"] = pd.to_datetime(tiempo_unique["hora"], format='%H:%M:%S').dt.hour

# Extraer la hora como un entero (1-24)
servicio["hora_solicita"] = servicio["hora_solicitud"].dt.hour

# Convertir 'fecha_solicitud' a datetime si aún no lo está
servicio["fecha_solicitud"] = pd.to_datetime(servicio["fecha_solicitud"])

# Crear la columna 'month_year' en formato 'YYYY-MM' para representar el año y el mes
servicio["año_mes"] = servicio["fecha_solicitud"].dt.to_period("M").astype(str)

servicio = servicio[["id","año_mes", "hora_solicita", "cliente_id", "mensajero_id"]]
servicio.fillna(-1, inplace=True)

# Convertir ambas columnas a tipo int para asegurar la coincidencia
print(hora["hora"].dtype)
# Convertir 'hora' en tiempo_unique al tipo datetime y extraer la hora como un entero


# Hacer el merge asegurando que coincidan
hecho_servicio_hora = pd.merge(servicio, tiempo_unique[["hora", "key_hora"]], left_on="hora_solicita", right_on="hora", how="left")
hecho_servicio_hora.head(5)

Index(['key_hora', 'hora', 'saved'], dtype='object')
object


,id,año_mes,hora_solicita,cliente_id,mensajero_id,hora,key_hora
0,34,2023-10,9,5,-1.0,9,9
1,35,2023-10,11,5,7.0,11,11
2,36,2023-10,19,5,-1.0,19,19
3,41,2023-11,9,5,-1.0,9,9
4,42,2023-11,9,5,-1.0,9,9


In [411]:
hecho_servicio_hora = pd.merge(hecho_servicio_hora, cliente[["cliente_id", "key_cliente"]], left_on="cliente_id", right_on="cliente_id", how="left")
hecho_servicio_hora.head(5)

,id,año_mes,hora_solicita,cliente_id,mensajero_id,hora,key_hora,key_cliente
0,34,2023-10,9,5,-1.0,9,9,7
1,35,2023-10,11,5,7.0,11,11,7
2,36,2023-10,19,5,-1.0,19,19,7
3,41,2023-11,9,5,-1.0,9,9,7
4,42,2023-11,9,5,-1.0,9,9,7


In [412]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
hecho_servicio_hora = hecho_servicio_hora.merge(mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
hecho_servicio_hora.fillna(-1, inplace=True)
hecho_servicio_hora.head(5)

,id,año_mes,hora_solicita,cliente_id,mensajero_id,hora,key_hora,key_cliente,key_mensajero
0,34,2023-10,9,5,-1.0,9,9,7,-1.0
1,35,2023-10,11,5,7.0,11,11,7,13.0
2,36,2023-10,19,5,-1.0,19,19,7,-1.0
3,41,2023-11,9,5,-1.0,9,9,7,-1.0
4,42,2023-11,9,5,-1.0,9,9,7,-1.0


In [413]:
print(len(hecho_servicio_hora))

28430


## Solicitudes_por_hora

In [414]:
# Contar solicitudes por cada 'año_mes'
solicitudes_por_hora = hecho_servicio_hora.groupby("hora").size().reset_index(name="solicitud_por_hora")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original

hecho_servicio_hora = hecho_servicio_hora.merge(solicitudes_por_hora, on="hora", how="left")

hecho_servicio_hora.head(5)




,id,año_mes,hora_solicita,cliente_id,mensajero_id,hora,key_hora,key_cliente,key_mensajero,solicitud_por_hora
0,34,2023-10,9,5,-1.0,9,9,7,-1.0,3391
1,35,2023-10,11,5,7.0,11,11,7,13.0,3295
2,36,2023-10,19,5,-1.0,19,19,7,-1.0,346
3,41,2023-11,9,5,-1.0,9,9,7,-1.0,3391
4,42,2023-11,9,5,-1.0,9,9,7,-1.0,3391


In [415]:
print(len(hecho_servicio_hora))

28430


## solicitud_hora_por_mes


In [416]:


solicitudes_por_hora_mes = hecho_servicio_hora.groupby(["año_mes", "hora_solicita"]).size().reset_index(name="solicitud_por_hora_mes")
hecho_servicio_hora = hecho_servicio_hora.merge(solicitudes_por_hora_mes, on=["año_mes", "hora_solicita"], how="left")

# Revisar los primeros datos ordenados
print(hecho_servicio_hora.head(10))


   id  año_mes  hora_solicita  cliente_id  mensajero_id  hora  key_hora  key_cliente  key_mensajero  solicitud_por_hora  solicitud_por_hora_mes
0  34  2023-10              9           5          -1.0     9         9            7           -1.0                3391                       1
1  35  2023-10             11           5           7.0    11        11            7           13.0                3295                       1
2  36  2023-10             19           5          -1.0    19        19            7           -1.0                 346                       3
3  41  2023-11              9           5          -1.0     9         9            7           -1.0                3391                       2
4  42  2023-11              9           5          -1.0     9         9            7           -1.0                3391                       2
5  43  2023-11             10           5          -1.0    10        10            7           -1.0                2932                 

## Solicitud_por_hora_cliente

In [417]:
print(hecho_servicio_hora.columns)

# Contar solicitudes por cada 'key_tiempo' y 'key_cliente' 
solicitud_por_hora_cliente = hecho_servicio_hora.groupby(["key_hora","key_cliente"]).size().reset_index(name="solicitud_por_hora_cliente")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_hora = hecho_servicio_hora.merge(solicitud_por_hora_cliente, on=["key_cliente", "key_hora"], how="left")
hecho_servicio_hora.head(5)

Index(['id', 'año_mes', 'hora_solicita', 'cliente_id', 'mensajero_id', 'hora', 'key_hora', 'key_cliente', 'key_mensajero', 'solicitud_por_hora', 'solicitud_por_hora_mes'], dtype='object')


,id,año_mes,hora_solicita,cliente_id,mensajero_id,hora,key_hora,key_cliente,key_mensajero,solicitud_por_hora,solicitud_por_hora_mes,solicitud_por_hora_cliente
0,34,2023-10,9,5,-1.0,9,9,7,-1.0,3391,1,660
1,35,2023-10,11,5,7.0,11,11,7,13.0,3295,1,862
2,36,2023-10,19,5,-1.0,19,19,7,-1.0,346,3,25
3,41,2023-11,9,5,-1.0,9,9,7,-1.0,3391,2,660
4,42,2023-11,9,5,-1.0,9,9,7,-1.0,3391,2,660


## Solicitud_por_mensajero

In [418]:
# Contar solicitudes por cada 'key_tiempo' y 'key_mensajero'
cantidad_solicitud_por_mensajero = hecho_servicio_hora.groupby(["key_mensajero"]).size().reset_index(name="cantidad_solicitud_por_mensajero")
# Hacer merge para agregar la columna 'solicitud_por_mes' al DataFrame original
hecho_servicio_mensual = hecho_servicio_hora.merge(cantidad_solicitud_por_mensajero, on=["key_mensajero"], how="left")
hecho_servicio_mensual.head(6)

,id,año_mes,hora_solicita,cliente_id,mensajero_id,hora,key_hora,key_cliente,key_mensajero,solicitud_por_hora,solicitud_por_hora_mes,solicitud_por_hora_cliente,cantidad_solicitud_por_mensajero
0,34,2023-10,9,5,-1.0,9,9,7,-1.0,3391,1,660,727
1,35,2023-10,11,5,7.0,11,11,7,13.0,3295,1,862,68
2,36,2023-10,19,5,-1.0,19,19,7,-1.0,346,3,25,727
3,41,2023-11,9,5,-1.0,9,9,7,-1.0,3391,2,660,727
4,42,2023-11,9,5,-1.0,9,9,7,-1.0,3391,2,660,727
5,43,2023-11,10,5,-1.0,10,10,7,-1.0,2932,1,360,727


## Eliminar filas

In [419]:
hecho_servicio_hora.drop(columns=["id","cliente_id","mensajero_id","hora"], inplace=True)
hecho_servicio_hora

,año_mes,hora_solicita,key_hora,key_cliente,key_mensajero,solicitud_por_hora,solicitud_por_hora_mes,solicitud_por_hora_cliente
0,2023-10,9,9,7,-1.0,3391,1,660
1,2023-10,11,11,7,13.0,3295,1,862
2,2023-10,19,19,7,-1.0,346,3,25
3,2023-11,9,9,7,-1.0,3391,2,660
4,2023-11,9,9,7,-1.0,3391,2,660
...,...,...,...,...,...,...,...,...
28425,2024-08,10,10,11,8.0,2932,478,2214
28426,2024-08,16,16,11,27.0,2124,365,1683
28427,2024-08,7,7,7,32.0,1066,150,293
28428,2024-08,10,10,11,18.0,2932,478,2214


## Load

In [420]:
hecho_servicio_hora.to_sql("hecho_servicio_hora", etl_conn, if_exists="replace", index_label="key_solicitud_servicio") 

430